In [ ]:
println("Start")

# Imports

In [2]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions.max

import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions.max


In [5]:
val df = Seq(
  (1,1, null, null, null ), 
  (2,1, null, null, null ), 
  (3,1, null, null, null ), 
  (4,1,"123.12", "124.52", "95.98" ), 
  (5,1, "452.12", "478.65", "1865.12" ), 
  (1,2,"2014.21", "147", "265"), 
  (2,2, "1457", "12483.00", "215.21"), 
  (3,2, null, null, null),
  (4,2, null, null, null) ).toDF("index", "category", "col1", "col2", "col3")

df: org.apache.spark.sql.DataFrame = [index: int, category: int ... 3 more fields]


In [6]:
df.show

+-----+--------+-------+--------+-------+
|index|category|   col1|    col2|   col3|
+-----+--------+-------+--------+-------+
|    1|       1|   null|    null|   null|
|    2|       1|   null|    null|   null|
|    3|       1|   null|    null|   null|
|    4|       1| 123.12|  124.52|  95.98|
|    5|       1| 452.12|  478.65|1865.12|
|    1|       2|2014.21|     147|    265|
|    2|       2|   1457|12483.00| 215.21|
|    3|       2|   null|    null|   null|
|    4|       2|   null|    null|   null|
+-----+--------+-------+--------+-------+



In [7]:
val cols = Seq("col1", "col2", "col3")

cols: Seq[String] = List(col1, col2, col3)


In [8]:
val beforeWindow = Window
    .partitionBy("category")
    .orderBy("index")
    .rangeBetween(Window.unboundedPreceding, Window.currentRow)

beforeWindow: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@1380abbe


In [9]:
val afterWindow = Window
    .partitionBy("category")
    .orderBy("index")
    .rangeBetween(Window.currentRow, Window.unboundedFollowing)

afterWindow: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@262f14d5


In [10]:
val result = cols.foldLeft(df)((updated, columnName) => 
        updated.withColumn(columnName,
                          coalesce(col(columnName),
                          last(columnName, ignoreNulls = true).over(beforeWindow),
                          first(columnName, ignoreNulls = true).over(afterWindow)
                          ))
)

result: org.apache.spark.sql.DataFrame = [index: int, category: int ... 3 more fields]


In [11]:
result.show(false)

+-----+--------+-------+--------+-------+
|index|category|col1   |col2    |col3   |
+-----+--------+-------+--------+-------+
|1    |1       |123.12 |124.52  |95.98  |
|2    |1       |123.12 |124.52  |95.98  |
|3    |1       |123.12 |124.52  |95.98  |
|4    |1       |123.12 |124.52  |95.98  |
|5    |1       |452.12 |478.65  |1865.12|
|1    |2       |2014.21|147     |265    |
|2    |2       |1457   |12483.00|215.21 |
|3    |2       |1457   |12483.00|215.21 |
|4    |2       |1457   |12483.00|215.21 |
+-----+--------+-------+--------+-------+

